In [10]:
# 연결 확인
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

1


In [2]:
# 종목 코드 가져오기
codemgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

kospi_code = codemgr.GetStockListByMarket(1) # 1: 코스피
kosdaq_code = codemgr.GetStockListByMarket(2) # 2: 코스닥

print(kospi_code[:5])
print(kosdaq_code[:5])

# 리스트로 형변환
kospi_code = list(kospi_code)
kosdaq_code = list(kosdaq_code)

('A000020', 'A000040', 'A000050', 'A000060', 'A000070')
('A000250', 'A000440', 'A001000', 'A001540', 'A001810')


In [3]:
stock_code = win32com.client.Dispatch('CpUtil.CpStockCode')
print(stock_code.GetCount())

3444


In [4]:
# CpStockCode를 사용하여 종목 코드를 종목명으로 나타내기
kospi_name = [stock_code.CodeToName(code) for code in kospi_code]
kosdaq_name = [stock_code.CodeToName(code) for code in kosdaq_code]

print(kospi_name[:5])
print(kosdaq_name[:5])

['동화약품', 'KR모터스', '경방', '메리츠화재', '삼양홀딩스']
['삼천당제약', '중앙에너비스', '신라섬유', '안국약품', '무림SP']


In [1]:
import time
instance = win32com.client.Dispatch("CpSysDib.StockChart")
path = 'C:/Users/sbum1/OneDrive/바탕 화면/Quant/kosdaq/'
col_name = ["날짜", "시간", "시가", "고가", "저가", "종가", "거래량"]

for code, name in zip(kosdaq_code, kosdaq_name):
    print(code, name)
    ## 파일 생성 (Pandas 설치에 오류가 나는 경우가 많아서 직접 라이팅)
    f = open(path + name +  ".csv", "w")
    f.write(",".join(col_name))
    f.write("\n")
    
    ## 데이터 요청
    instance.SetInputValue(0, code)  # 종목코드
    instance.SetInputValue(1, ord('1')) # 분명 cybos에 분/틱 데이터는 기간으로 요청이 안 된다 하였는데 잘 됨.
    instance.SetInputValue(3, 20191005)
    instance.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])  # 날짜, 시가, 고가, 저가, 종가, 거래량
    instance.SetInputValue(6, ord('m'))  # 차트 주기 - 일간 차트
    instance.SetInputValue(9, ord('1'))  # 수정주가 사용
    while True:
        instance.BlockRequest()
        numData = instance.GetHeaderValue(3)
        time.sleep(0.28) # 15초간 60건 이상 호출 시 락걸리기에 0.28초로 설정

        for row_ind in range(numData):
            record = [] # record: 데이터 내 하나의 로우 / 초기화하여 컬럼 값을 가져오고 이를 data에 추가
            for col_ind in range(len(col_name)): 
                record.append(instance.GetDataValue(col_ind, row_ind))

            f.write(','.join(list(map(str, record)))) # record에 있는 모든 요소를 문자열로 만들어서 조인하여 라이팅

            if row_ind != numData - 1: # 마지막에는 줄바꿈을 입력하지 않음
                f.write('\n')
                
        if instance.Continue == False :
            break

    f.close()

NameError: name 'win32com' is not defined

In [38]:
import time
instance = win32com.client.Dispatch("CpSysDib.StockChart")
path = 'C:/Users/sbum1/OneDrive/바탕 화면/Quant/Tic/kosdaq/'
col_name = ["날짜", "시간", "시가", "고가", "저가", "종가", "거래량"]
kosdaq_ticerror = []

for code, name in zip(kosdaq_code, kosdaq_name):
    print(code, name)
    if '/' in name:
        kosdaq_ticerror.append(name)
    else:
    ## 파일 생성 (Pandas 설치에 오류가 나는 경우가 많아서 직접 라이팅)
        f = open(path + name +  ".csv", "w")
        f.write(",".join(col_name))
        f.write("\n")

    ## 데이터 요청
        instance.SetInputValue(0, code)  # 종목코드
        instance.SetInputValue(1, ord('1'))
        instance.SetInputValue(3, 20210824)
        instance.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])  # 날짜, 시가, 고가, 저가, 종가, 거래량
        instance.SetInputValue(6, ord('T'))  # 차트 주기 - 일간 차트
        instance.SetInputValue(9, ord('1'))  # 수정주가 사용
        while True:
            instance.BlockRequest()
            numData = instance.GetHeaderValue(3)
            time.sleep(0.28)

            for row_ind in range(numData):
                record = [] # record: 데이터 내 하나의 로우 / 초기화하여 컬럼 값을 가져오고 이를 data에 추가
                for col_ind in range(len(col_name)): 
                    record.append(instance.GetDataValue(col_ind, row_ind))

                f.write(','.join(list(map(str, record)))) # record에 있는 모든 요소를 문자열로 만들어서 조인하여 라이팅

                if row_ind != numData - 1: # 마지막에는 줄바꿈을 입력하지 않음
                    f.write('\n')
            if instance.Continue == False :
                break

        f.close()
    

A000250 삼천당제약
A000440 중앙에너비스
A001000 신라섬유
A001540 안국약품
A001810 무림SP
A001840 이화공영
A002230 피에스텍
A002290 삼일기업공사
A002680 한탑
A002800 신신제약
A003100 선광
A003310 대주산업
A003380 하림지주
A003800 에이스침대
A004590 한국가구
A004650 창해에탄올
A004780 대륙제관
A005160 동국산업
A005290 동진쎄미켐
A005670 푸드웰
A005710 대원산업
A005860 한일사료
A005990 매일홀딩스
A006050 국영지앤엠
A006140 피제이전자
A006580 대양제지
A006620 동구바이오제약
A006730 서부T&D
A006910 보성파워텍
A006920 모헨즈
A007330 푸른저축은행
A007370 진양제약
A007390 네이처셀
A007530 영신금속
A007680 대원
A007720 대명소노시즌
A007770 한일화학
A007820 에스엠코어
A008290 원풍물산
A008370 원풍
A008470 부스타
A008830 대동기어
A009300 삼아제약
A009520 포스코엠텍
A009620 삼보산업
A009730 코센
A009780 엠에스씨
A010170 대한광통신
A010240 흥국
A010280 쌍용정보통신
A010470 오리콤
A011040 경동제약
A011080 형지I&C
A011320 유니크
A011370 서한
A011560 세보엠이씨
A012340 뉴인텍
A012620 원일특강
A012700 리드코프
A012790 신일제약
A012860 모베이스전자
A013030 하이록코리아
A013120 동원개발
A013310 아진산업
A013720 CBI
A013810 스페코
A013990 아가방컴퍼니
A014100 메디앙스
A014190 원익큐브
A014200 광림
A014470 부방
A014570 고려제약
A014620 성광벤드
A014940 오리엔탈정공
A014970 삼륭물산
A015710 코콤
A0157

A071200 인피니트헬스케어
A071280 로체시스템즈
A071460 위니아딤채
A071670 에이테크솔루션
A071850 캐스텍코리아
A072020 중앙백신
A072470 우리산업홀딩스
A072520 제넨바이오
A072770 율호
A072870 메가스터디
A072950 빛샘전자
A072990 에이치시티
A073010 케이에스피
A073110 엘엠에스
A073190 듀오백
A073490 이노와이어리스
A073540 에프알텍
A073560 우리손에프앤지
A073570 WI
A073640 테라사이언스
A074430 아미노로직스
A074600 원익QnC
A075130 플랜티넷
A075970 동국알앤에스
A076080 웰크론한텍
A076610 해성옵틱스
A077360 덕산하이메탈
A078020 이베스트투자증권
A078070 유비쿼스홀딩스
A078130 국일제지
A078140 대봉엘에스
A078150 HB테크놀러지
A078160 메디포스트
A078340 컴투스
A078350 한양디지텍
A078590 OQP
A078600 대주전자재료
A078650 지나인제약
A078860 아이오케이
A078890 가온미디어
A078940 코드네이처
A079000 와토스코리아
A079170 한창산업
A079190 EMW
A079370 제우스
A079650 서산
A079810 디이엔티
A079940 가비아
A079950 인베니아
A079960 동양이엔피
A079970 투비소프트
A080000 에스엔유
A080010 이상네트웍스
A080160 모두투어
A080220 제주반도체
A080420 모다이노칩
A080470 성창오토텍
A080520 오디텍
A080530 코디
A080580 오킨스전자
A080720 한국유니온제약
A081150 티플랙스
A081580 성우전자
A082210 옵트론텍
A082270 젬백스
A082660 코스나인
A082800 비보존 헬스케어
A082850 우리바이오
A082920 비츠로셀
A083310 엘오티베큠
A083450 GST
A083470 이엠앤아이
A08350

A241520 DSC인베스트먼트
A241690 유니테크노
A241710 코스메카코리아
A241770 메카로
A241790 오션브릿지
A241820 피씨엘
A241840 에이스토리
A242040 나무기술
A243070 휴온스
A243840 신흥에스이씨
A244460 올리패스
A245620 EDGC
A246690 TS인베스트먼트
A246710 티앤알바이오팹
A246720 아스타
A246960 이노테라피
A247540 에코프로비엠
A247660 나노씨엠에스
A250000 보라티알
A250060 모비스
A250930 예선테크
A251370 와이엠티
A251630 브이원텍
A251970 펌텍코리아
A252500 세화피앤씨
A252990 샘씨엔에스
A253450 스튜디오드래곤
A253590 네오셈
A253840 수젠텍
A254120 자비스
A255220 SG
A255440 야스
A256150 한독크린텍
A256630 포인트엔지니어링
A256840 한국비엔씨
A256940 케이피에스
A257370 명성티엔에스
A258610 이더블유케이
A258790 소프트캠프
A258830 세종메디칼
A259630 엠플러스
A260660 알리코제약
A260930 씨티케이
A261200 덴티스
A262260 에이프로
A262840 아이퀘스트
A263020 디케이앤디
A263050 유틸렉스
A263540 샘코
A263600 덕우전자
A263690 디알젬
A263700 케어랩스
A263720 디앤씨미디어
A263750 펄어비스
A263770 유에스티
A263800 데이타솔루션
A263810 상신전자
A263860 지니언스
A263920 휴온스블러썸
A264450 유비쿼스
A264660 씨앤지하이테크
A264850 이랜시스
A265520 AP시스템
A265560 영화테크
A265740 엔에프씨
A267320 나인테크
A267790 배럴
A267980 매일유업
A268600 셀리버리
A269620 시스웍
A270520 지엔원에너지
A270660 에브리봇
A270870 뉴트리
A272110 케이엔제